# 📘 Big Data สัปดาห์ที่ 4: Distributed Processing & Apache Spark (Deep Dive)


**สัปดาห์ที่:** 4  

> 🎯 **เป้าหมายวันนี้:** เข้าใจแนวคิด Distributed Computing, สถาปัตยกรรมของ Apache Spark, Lazy Evaluation, DAG และ Shuffle พร้อมลงมือปฏิบัติจริงด้วย PySpark

**ใช้ได้ทั้ง**: Google Colab (แนะนำ) / Local Jupyter  

---


# 1. 🖥️ ทำไมเครื่องเดียวถึงไม่พอ? (Why Single Machine Fails)

## ปัญหาของการประมวลผลด้วยเครื่องเดียว

เมื่อข้อมูลมีขนาดใหญ่ขึ้น เราจะพบข้อจำกัดหลายด้าน:

| ข้อจำกัด | คำอธิบาย | ตัวอย่างปัญหา |
|----------|----------|---------------|
| **RAM** | หน่วยความจำมีจำกัด (เช่น 16 GB) | ลองโหลด CSV 50 GB ด้วย Pandas → MemoryError |
| **CPU** | จำนวน Core มีจำกัด (เช่น 4-8 Cores) | งาน groupBy บน 1 พันล้านแถว ใช้เวลาหลายชั่วโมง |
| **I/O** | Disk อ่าน/เขียนช้า | สแกนไฟล์ 100 GB จาก HDD ใช้เวลา ~15 นาที |
| **ไม่ขยายได้** | เพิ่ม RAM/CPU มีเพดาน | เครื่องแพงสุดในตลาดก็มีขีดจำกัด |

### 💡 แนวคิดสำคัญ (Key Insight)

> เมื่อ **ขนาดข้อมูล × ความซับซ้อน > ขีดความสามารถของเครื่องเดียว**  
> → ต้องใช้ **Distributed Computing** (การประมวลผลแบบกระจาย)

### เปรียบเทียบง่าย ๆ

- **เครื่องเดียว** = คนเดียวยกอิฐ 10,000 ก้อน → เหนื่อย ช้า เสียเวลา
- **Distributed** = คน 100 คน ช่วยกันยกคนละ 100 ก้อน → เสร็จเร็ว!


<div align="center">
  <img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/single_vs_distributed.png?raw=1" width="800" alt="Single Machine vs Distributed Computing: คนเดียวแบกของหนัก vs หลายคนช่วยกันแบก">
  <br><i>Single Machine vs Distributed Computing: คนเดียวแบกของหนัก vs หลายคนช่วยกันแบก</i>
</div>


In [ ]:
# ตัวอย่าง: ลองดูข้อจำกัดของเครื่องเดียว
import psutil, os

ram_gb = psutil.virtual_memory().total / (1024**3)
cpu_count = os.cpu_count()

print(f"🖥️ เครื่องนี้มี RAM: {ram_gb:.1f} GB")
print(f"🔧 จำนวน CPU Cores: {cpu_count}")
print(f"\n💡 ถ้าข้อมูลใหญ่กว่า {ram_gb:.0f} GB → Pandas จะ crash!")
print(f"💡 ถ้า query ซับซ้อน ใช้ได้แค่ {cpu_count} cores → ช้า!")


### ✏️ แบบฝึกหัดที่ 1: คำนวณขีดจำกัด

**โจทย์:** บริษัทมีข้อมูล Log 500 GB/วัน ต้อง JOIN กับตาราง 50 GB และ GROUP BY  
เครื่อง Server มี RAM 64 GB, 16 Cores

**คำถาม** (เติมคำตอบในช่องว่าง):


In [22]:
# แบบฝึกหัดที่ 1: เติมคำตอบ

# 1) ข้อมูลรวมทั้งหมดกี่ GB?
total_data_gb = 550  # เติมตัวเลข

# 2) RAM เพียงพอหรือไม่? (True/False)
ram_enough = False  # True หรือ False

# 3) ถ้าอ่าน disk ได้ 200 MB/s จะใช้เวลานานกี่วินาที (ประมาณ)?
read_time_sec = 2750  # เติมตัวเลข

# 4) ควรใช้ Distributed Computing หรือไม่? เพราะอะไร?
answer = "Yes"

print(f"ข้อมูลรวม: {total_data_gb} GB")
print(f"RAM เพียงพอ: {ram_enough}")
print(f"เวลาอ่าน disk: {read_time_sec} วินาที ≈ {read_time_sec/60:.1f} นาที")
print(f"คำตอบ: {answer}")


ข้อมูลรวม: 550 GB
RAM เพียงพอ: False
เวลาอ่าน disk: 2750 วินาที ≈ 45.8 นาที
คำตอบ: Yes


# 2. 🔥 Apache Spark คืออะไร?

## นิยาม

Apache Spark คือ **Distributed Computing Engine** (เครื่องยนต์ประมวลผลแบบกระจาย) สำหรับข้อมูลขนาดใหญ่

> Spark เหมือน "สมองกลาง" ที่สั่งการให้เครื่องหลายร้อยตัวทำงานพร้อมกัน

## คุณสมบัติหลัก

| คุณสมบัติ | คำอธิบาย | เปรียบเทียบ |
|-----------|----------|-------------|
| **In-Memory Processing** | ประมวลผลใน RAM ไม่ต้องเขียน disk ทุกขั้นตอน | เร็วกว่า Hadoop MapReduce 10-100 เท่า |
| **DAG Execution** | สร้างแผนงานเป็น Graph ก่อนทำจริง | เหมือนวางแผนก่อนสร้างบ้าน |
| **Fault Tolerance** | ถ้าเครื่องพัง สามารถคำนวณใหม่ได้ | เหมือนมีสำเนาสูตรอาหาร |
| **Unified Engine** | รองรับ SQL, ML, Streaming, Graph | เครื่องมือเดียวทำได้หลายอย่าง |

## ⚠️ Spark ไม่ใช่อะไร?

- ❌ **ไม่ใช่ Database** — Spark ไม่เก็บข้อมูลถาวร
- ❌ **ไม่ใช่ Storage System** — Spark อ่านข้อมูลจาก HDFS, S3, Parquet ฯลฯ
- ✅ **Spark = Compute Layer** — เป็น "ชั้นประมวลผล" ที่อยู่ระหว่างข้อมูลกับผลลัพธ์

### เปรียบเทียบ Spark กับเครื่องมืออื่น

| เครื่องมือ | เหมาะกับ | ขนาดข้อมูล | ความเร็ว |
|-----------|---------|-----------|---------|
| **Pandas** | เครื่องเดียว, EDA | MB - ไม่เกิน RAM | เร็ว (ข้อมูลเล็ก) |
| **DuckDB** | SQL Analytics เครื่องเดียว | MB - GB | เร็วมาก |
| **Spark** | Distributed, Data Pipeline | GB - PB | เร็ว (ข้อมูลใหญ่) |
| **Hadoop MR** | Batch บน HDFS | TB - PB | ช้า (เขียน disk ทุกขั้น) |


# 3. 🏗️ สถาปัตยกรรม Spark เชิงลึก (Spark Architecture Deep Dive)

## ส่วนประกอบหลัก 3 ส่วน

```
┌─────────────────────────────────────┐
│         Cluster Manager             │
│    (ผู้จัดสรรทรัพยากร: YARN/K8s)     │
└──────────┬──────────────────────────┘
           │ จัดสรร CPU/RAM
    ┌──────┴──────┐
    ▼             ▼
┌────────┐   ┌────────────────────┐
│ Driver │──▶│   Executors (N ตัว) │
│ (สมอง) │   │   (แรงงาน)         │
└────────┘   └────────────────────┘
```

### 🧠 Driver (สมอง) — หน้าที่:
1. **สร้าง SparkSession** — เปิดประตูเข้าสู่ Spark
2. **สร้าง Logical Plan** — แปลงโค้ดเป็นแผนงาน
3. **Optimize** — ปรับแผนให้เร็วที่สุด (ผ่าน Catalyst Optimizer)
4. **แบ่งงาน** — แบ่ง Job → Stages → Tasks
5. **ติดตามผล** — ดูว่า Executor ทำเสร็จหรือยัง

### ⚙️ Executor (แรงงาน) — หน้าที่:
1. **รับ Task มาทำ** — ประมวลผลข้อมูลจริง ๆ
2. **เก็บ Cache** — เก็บข้อมูลใน RAM สำหรับใช้ซ้ำ
3. **Shuffle** — ส่งข้อมูลข้ามเครื่องเมื่อจำเป็น
4. **รายงานผล** — ส่งผลกลับไปที่ Driver

### 🔄 ลำดับการทำงาน: Job → Stage → Task → Partition

| ระดับ | คำอธิบาย | เกิดจาก |
|------|----------|--------|
| **Job** | งานใหญ่ 1 ชิ้น | เกิดจาก Action (เช่น `.count()`) |
| **Stage** | ช่วงงานที่ทำได้โดยไม่ต้อง Shuffle | เกิดจากการตัดที่จุด Shuffle |
| **Task** | หน่วยงานย่อยที่สุด | 1 Task ประมวลผล 1 Partition |
| **Partition** | ชิ้นส่วนข้อมูล | ไฟล์ถูกแบ่งเป็น Partition อัตโนมัติ |

> 💡 **เปรียบเทียบ:** Job = สร้างบ้าน, Stage = ขั้นตอน (เทฐาน/ก่อผนัง/มุงหลังคา), Task = ช่างแต่ละคน, Partition = วัสดุที่แบ่งให้ช่างแต่ละคน


<div align="center">
  <img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/driver_internal.png?raw=1" width="800" alt="Driver Internals Diagram">
  <br><i>โครงสร้างภายในของ Driver: SparkSession, DAGScheduler, TaskScheduler</i>
</div>

#### 🧠 เจาะลึก Driver Internals
- **SparkSession**: จุดเริ่มต้นของการเขียนโค้ด (Entry Point)
- **DAGScheduler**: แปลง Logical Plan เป็น Physical Plan และแบ่งงานเป็น Stages (Stage 1, Stage 2)
- **TaskScheduler**: รับ Stage มาแตกเป็น Task ย่อยๆ แล้วส่งไปให้ Executor ที่ว่างอยู่
- **SchedulerBackend**: สื่อสารกับ Cluster Manager เพื่อขอทรัพยากร (CPU/RAM)


<div align="center">
  <img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/spark_chef_analogy.png?raw=1" width="800" alt="Spark Architecture Analogy: Driver (Chef) สั่งงาน Executors (Cooks)">
  <br><i>Spark Architecture Analogy: Driver (Chef) สั่งงาน Executors (Cooks)</i>
</div>


### 3.1 🧠 Deep Dive: SparkSession vs SparkContext

หลายคนอาจสงสัยว่า `SparkSession` กับ `SparkContext` ต่างกันอย่างไร?

#### 1. SparkContext (`sc`) — The Engine 🔧
- เป็น **Entry Point** ดั้งเดิมของ Spark (ตั้งแต่เวอร์ชัน 1.x)
- ทำหน้าที่เชื่อมต่อกับ Cluster Manager (YARN, K8s, Standalone)
- จัดการ Executors, Memory, และ Job Scheduling
- **เปรียบเทียบ:** เหมือน **"เครื่องยนต์"** ของรถยนต์ — ทำหน้าที่ขับเคลื่อนกลไกภายใน

#### 2. SparkSession (`spark`) — The Dashboard 🚗
- เริ่มใช้ใน Spark 2.0+
- เป็น **Unified Entry Point** ที่รวมทุกอย่างไว้ในที่เดียว:
  - `SparkContext` (Core)
  - `SQLContext` (DataFrames/SQL)
  - `HiveContext` (Hive tables)
  - `StreamingContext` (Streaming)
- **เปรียบเทียบ:** เหมือน **"คนขับ"** หรือ **"แผงหน้าปัด"** ที่เราใช้งานจริง (เราขับรถผ่านพวงมาลัย ไม่ได้ไปหมุนล้อเอง)

> 💡 **Best Practice:** ในโค้ดปัจจุบัน **เราจะใช้ `SparkSession` เสมอ** (แต่เบื้องหลังมันก็เรียกใช้ `SparkContext` อยู่ดี)


## 🔬 Lab: เริ่มต้นใช้งาน Spark

### Step 1: ติดตั้งและสร้าง SparkSession


In [1]:
# ติดตั้ง PySpark (สำหรับ Colab)
!pip -q install pyspark

from pyspark.sql import SparkSession
import os

# สร้าง SparkSession — ประตูเข้าสู่ Spark
# 💡 config("spark.ui.port", "4050") กำหนด port เพื่อเลี่ยง port 4040 ที่อาจชนกัน
spark = SparkSession.builder \
    .appName("BigData-Week4-DeepDive") \
    .config("spark.driver.memory", "2g") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

# เข้าถึง SparkContext ที่ถูก wrap อยู่ข้างใน
sc = spark.sparkContext

print(f"✅ Spark Version: {spark.version}")
print(f"📱 App Name: {sc.appName}")
print(f"🖥️ Master: {sc.master}")


✅ Spark Version: 4.0.2
📱 App Name: BigData-Week4-DeepDive
🖥️ Master: local[*]


### 3.2 🛠️ แก้ปัญหา: เปิด Spark UI ใน Google Colab

**ปัญหา:** ปกติ Spark UI จะอยู่ที่ `localhost:4040` แต่ Google Colab รันบนเครื่อง Server (Virtual Machine) ทำให้เราเข้าถึง `localhost` ไม่ได้โดยตรง

**วิธีแก้:** เราต้องใช้ library `google.colab.output` เพื่อสร้าง Proxy ให้เราเปิดหน้าเว็บได้

รัน cell นี้เพื่อเปิด Spark UI 👇


In [4]:
# 🛠️ Code สำหรับเปิด Spark UI ใน Colab
try:
    from google.colab import output

    # ฟังก์ชันสำหรับเปิด Spark UI
    def show_spark_ui(port=4050):
        # ดึง URL ของ Spark UI ผ่าน Proxy ของ Colab
        url = output.serve_kernel_port_as_iframe(port)
        # หรือแสดงเป็น Link ให้คลิก (ถ้า iframe เล็กไป)
        # output.serve_kernel_port_as_window(port)
        print(f"🚀 Spark UI เปิดอยู่ที่: {url}")
        print("💡 ถ้าไม่เห็นผลลัพธ์ ให้ลองคลิก link นี้แทน (อาจต้อง login Google):")
        output.serve_kernel_port_as_window(port, path='/jobs/')

    # เรียกใช้งาน (Default port ที่เราตั้งไว้คือ 4050)
    show_spark_ui(4050)
except ImportError:
    print("⚠️ ไม่ได้รันบน Google Colab หรือไม่พบ library google.colab")
    print(f"👉 ถ้าคุณรันบน Local Jupyter ให้เปิด: http://localhost:4050")


<IPython.core.display.Javascript object>

🚀 Spark UI เปิดอยู่ที่: None
💡 ถ้าไม่เห็นผลลัพธ์ ให้ลองคลิก link นี้แทน (อาจต้อง login Google):
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

# 4. 🏢 Cluster Managers (ผู้จัดสรรทรัพยากร)

Cluster Manager ทำหน้าที่จัดสรร CPU และ RAM ให้ Spark ใช้งาน เปรียบได้กับ "ผู้จัดการอาคาร" ที่จัดห้องให้ผู้เช่า

## เปรียบเทียบ Cluster Manager 3 แบบ

| | **Standalone** | **YARN** | **Kubernetes** |
|---|---|---|---|
| **ความหมาย** | Cluster Manager ของ Spark เอง | Hadoop ecosystem | Container-based |
| **ติดตั้ง** | ง่ายที่สุด | ต้องมี Hadoop | ต้องมี K8s cluster |
| **เหมาะกับ** | ทดลอง, ทีมเล็ก | องค์กรใหญ่ | Cloud-native |
| **ข้อดี** | Simple, เร็ว | Queue management, แชร์ resource | Auto-scaling, Isolate |
| **ตัวอย่าง** | Spark Standalone | AWS EMR, CDH | GKE, EKS |

> 💡 **ในคอร์สนี้** เราใช้ `local[*]` ซึ่งหมายถึง "รันบนเครื่องเดียว ใช้ทุก CPU core" — เหมาะสำหรับเรียนรู้


In [3]:
# ดูว่าเรากำลังใช้ Cluster Manager แบบไหน
print(f"🏢 Master URL: {spark.sparkContext.master}")
print()
print("📝 ความหมายของ Master URL:")
print("  local     = เครื่องเดียว, 1 thread")
print("  local[*]  = เครื่องเดียว, ใช้ทุก core")
print("  local[4]  = เครื่องเดียว, 4 threads")
print("  yarn      = ใช้ YARN บน Hadoop cluster")
print("  k8s://... = ใช้ Kubernetes cluster")
print("  spark://  = ใช้ Standalone cluster")


🏢 Master URL: local[*]

📝 ความหมายของ Master URL:
  local     = เครื่องเดียว, 1 thread
  local[*]  = เครื่องเดียว, ใช้ทุก core
  local[4]  = เครื่องเดียว, 4 threads
  yarn      = ใช้ YARN บน Hadoop cluster
  k8s://... = ใช้ Kubernetes cluster
  spark://  = ใช้ Standalone cluster


# 5. ⏳ Lazy Evaluation (แนวคิดเชิงลึก)

## หลักการสำคัญที่สุดของ Spark

> **Spark ไม่ทำงานทันที!** Spark จะ "จดบันทึก" ว่าเราต้องการอะไร แล้วค่อยทำตอนที่จำเป็นจริง ๆ

### เปรียบเทียบ

| | Pandas (Eager) | Spark (Lazy) |
|---|---|---|
| **เมื่อเขียนคำสั่ง** | ทำทันที | จดไว้ก่อน (สร้าง Plan) |
| **ข้อดี** | เห็นผลเลย | ปรับแผนให้เร็วที่สุดก่อนทำ |
| **ข้อเสีย** | ไม่ optimize ข้ามขั้นตอน | ต้อง trigger ด้วย Action |

### Transformation vs Action

#### 🔹 Transformations (Lazy — จดไว้ก่อน ยังไม่ทำ)
| คำสั่ง | หน้าที่ |
|--------|--------|
| `select()` | เลือกคอลัมน์ |
| `filter()` / `where()` | กรองแถว |
| `groupBy()` | จัดกลุ่ม |
| `join()` | เชื่อมตาราง |
| `withColumn()` | เพิ่ม/แก้คอลัมน์ |
| `orderBy()` | เรียงลำดับ |

#### 🔸 Actions (ตัวกระตุ้น — สั่งให้ทำจริง!)
| คำสั่ง | หน้าที่ |
|--------|--------|
| `show()` | แสดงข้อมูล |
| `count()` | นับจำนวนแถว |
| `collect()` | ดึงข้อมูลทั้งหมดมาที่ Driver |
| `write()` | เขียนข้อมูลลง disk |
| `take(n)` | ดึง n แถวแรก |


<div align="center">
  <img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/lazy_evaluation.png?raw=1" width="800" alt="Lazy Evaluation: จดออร์เดอร์ (Transformation) vs เสิร์ฟอาหาร (Action)">
  <br><i>Lazy Evaluation: จดออร์เดอร์ (Transformation) vs เสิร์ฟอาหาร (Action)</i>
</div>


### 🔬 Lab: พิสูจน์ Lazy Evaluation


In [5]:
# สร้างข้อมูลตัวอย่าง
data = [(i, f"name_{i}", i * 100, "A" if i % 2 == 0 else "B")
        for i in range(1, 10001)]
df = spark.createDataFrame(data, ["id", "name", "salary", "dept"])

print(f"✅ สร้าง DataFrame ขนาด {df.count()} แถว")
df.show(5)


✅ สร้าง DataFrame ขนาด 10000 แถว
+---+------+------+----+
| id|  name|salary|dept|
+---+------+------+----+
|  1|name_1|   100|   B|
|  2|name_2|   200|   A|
|  3|name_3|   300|   B|
|  4|name_4|   400|   A|
|  5|name_5|   500|   B|
+---+------+------+----+
only showing top 5 rows


In [6]:
import time

# ===== Transformation (Lazy) — ไม่มีอะไรเกิดขึ้น! =====
print("🔹 เริ่มเขียน Transformation...")
t0 = time.time()

df2 = df.select("id", "salary", "dept")      # เลือกคอลัมน์
df3 = df2.filter(df2["salary"] > 500)         # กรอง
df4 = df3.groupBy("dept").count()             # จัดกลุ่ม

t1 = time.time()
print(f"⏱️ เวลาที่ใช้ (Transformation): {t1-t0:.4f} วินาที")
print("💡 สังเกต: เร็วมาก เพราะ Spark ยังไม่ได้ทำอะไรเลย!")

# ===== Action (Trigger!) — ตอนนี้ Spark ทำงานจริง! =====
print("\n🔸 เรียก Action (.show()) — Spark เริ่มทำงาน!")
t2 = time.time()

df4.show()

t3 = time.time()
print(f"⏱️ เวลาที่ใช้ (Action): {t3-t2:.4f} วินาที")
print("💡 สังเกต: ช้ากว่า เพราะ Spark ต้องทำ Transformation ทั้งหมดตอนนี้!")


🔹 เริ่มเขียน Transformation...
⏱️ เวลาที่ใช้ (Transformation): 0.1538 วินาที
💡 สังเกต: เร็วมาก เพราะ Spark ยังไม่ได้ทำอะไรเลย!

🔸 เรียก Action (.show()) — Spark เริ่มทำงาน!
+----+-----+
|dept|count|
+----+-----+
|   B| 4997|
|   A| 4998|
+----+-----+

⏱️ เวลาที่ใช้ (Action): 1.9786 วินาที
💡 สังเกต: ช้ากว่า เพราะ Spark ต้องทำ Transformation ทั้งหมดตอนนี้!


### ✏️ แบบฝึกหัดที่ 2: Transformation vs Action

**โจทย์:** จากโค้ดด้านล่าง ระบุว่าบรรทัดไหนเป็น Transformation (T) หรือ Action (A)


In [7]:
# แบบฝึกหัดที่ 2: เติม T (Transformation) หรือ A (Action)

# บรรทัดที่ 1: df.select("name", "salary")      →  ________
# บรรทัดที่ 2: df.filter(df["salary"] > 1000)    →  ________
# บรรทัดที่ 3: df.count()                        →  ________
# บรรทัดที่ 4: df.groupBy("dept").avg("salary")  →  ________
# บรรทัดที่ 5: df.show(10)                       →  ________
# บรรทัดที่ 6: df.orderBy("salary")              →  ________
# บรรทัดที่ 7: df.collect()                      →  ________
# บรรทัดที่ 8: df.withColumn("bonus", df["salary"] * 0.1)  →  ________

answers = {
    1: "T",
    2: "t",
    3: "a",
    4: "t",
    5: "a",
    6: "t",
    7: "a",
    8: "a",
}

for k, v in answers.items():
    print(f"บรรทัดที่ {k}: {v}")


บรรทัดที่ 1: T
บรรทัดที่ 2: t
บรรทัดที่ 3: a
บรรทัดที่ 4: t
บรรทัดที่ 5: a
บรรทัดที่ 6: t
บรรทัดที่ 7: a
บรรทัดที่ 8: a


# 6. 📊 DAG (Directed Acyclic Graph)

## DAG คืออะไร?

DAG คือ **กราฟ** แสดงลำดับการทำงานของ Spark โดย:
- **Directed** = มีทิศทาง (ทำจากซ้ายไปขวา)
- **Acyclic** = ไม่วนซ้ำ (ไม่กลับไปทำซ้ำ)
- **Graph** = แผนภาพเชื่อมโยง

```
Read CSV → Filter → Select → GroupBy → [Shuffle] → Aggregate → Show
  Stage 1 ──────────────────────▶  Stage 2 ────────────────▶
```

## ขั้นตอนการสร้างแผน

1. **Logical Plan** — แปลงจากโค้ดที่เราเขียน
2. **Optimized Logical Plan** — Catalyst Optimizer ปรับปรุงให้ดีขึ้น
3. **Physical Plan** — แผนจริงที่จะทำ (เลือกอัลกอริทึมที่เร็วที่สุด)

> 💡 เปรียบเทียบ: เหมือน GPS — เราบอกจุดหมาย (Logical) → GPS คำนวณเส้นทางที่ดีที่สุด (Optimized) → แสดงเส้นทางจริง (Physical)


In [8]:
# ดู Execution Plan ของ Spark
df_plan = df.select("id", "salary", "dept") \
            .filter(df["salary"] > 500) \
            .groupBy("dept") \
            .avg("salary")

print("📋 Execution Plan:")
print("=" * 50)
df_plan.explain("formatted")


📋 Execution Plan:
== Physical Plan ==
AdaptiveSparkPlan (7)
+- HashAggregate (6)
   +- Exchange (5)
      +- HashAggregate (4)
         +- Project (3)
            +- Filter (2)
               +- Scan ExistingRDD (1)


(1) Scan ExistingRDD
Output [4]: [id#0L, name#1, salary#2L, dept#3]
Arguments: [id#0L, name#1, salary#2L, dept#3], MapPartitionsRDD[4] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:0, ExistingRDD, UnknownPartitioning(0)

(2) Filter
Input [4]: [id#0L, name#1, salary#2L, dept#3]
Condition : (isnotnull(salary#2L) AND (salary#2L > 500))

(3) Project
Output [2]: [salary#2L, dept#3]
Input [4]: [id#0L, name#1, salary#2L, dept#3]

(4) HashAggregate
Input [2]: [salary#2L, dept#3]
Keys [1]: [dept#3]
Functions [1]: [partial_avg(salary#2L)]
Aggregate Attributes [2]: [sum#44, count#45L]
Results [3]: [dept#3, sum#46, count#47L]

(5) Exchange
Input [3]: [dept#3, sum#46, count#47L]
Arguments: hashpartitioning(dept#3, 200), ENSURE_REQUIREMENTS, [plan_id=115]

(6) HashAggregat

In [9]:
# เปรียบเทียบ: ดูแผนแบบละเอียด (extended)
print("📋 Extended Plan (ดู Logical → Physical):")
print("=" * 50)
df_plan.explain("extended")


📋 Extended Plan (ดู Logical → Physical):
== Parsed Logical Plan ==
'Aggregate ['dept], ['dept, unresolvedalias('avg(salary#2L))]
+- Filter (salary#2L > cast(500 as bigint))
   +- Project [id#0L, salary#2L, dept#3]
      +- LogicalRDD [id#0L, name#1, salary#2L, dept#3], false

== Analyzed Logical Plan ==
dept: string, avg(salary): double
Aggregate [dept#3], [dept#3, avg(salary#2L) AS avg(salary)#43]
+- Filter (salary#2L > cast(500 as bigint))
   +- Project [id#0L, salary#2L, dept#3]
      +- LogicalRDD [id#0L, name#1, salary#2L, dept#3], false

== Optimized Logical Plan ==
Aggregate [dept#3], [dept#3, avg(salary#2L) AS avg(salary)#43]
+- Project [salary#2L, dept#3]
   +- Filter (isnotnull(salary#2L) AND (salary#2L > 500))
      +- LogicalRDD [id#0L, name#1, salary#2L, dept#3], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[dept#3], functions=[avg(salary#2L)], output=[dept#3, avg(salary)#43])
   +- Exchange hashpartitioning(dept#3, 200), ENSURE_REQU

# 7. 🔀 Shuffle — การส่งข้อมูลข้ามเครื่อง

## Shuffle คืออะไร?

Shuffle คือการ **ส่งข้อมูลข้ามเครื่อง** (หรือข้าม partition) เมื่อ Spark ต้องจัดกลุ่มข้อมูลใหม่

### เมื่อไหร่เกิด Shuffle?

| คำสั่ง | ทำไมต้อง Shuffle | ตัวอย่าง |
|--------|-----------------|---------|
| `groupBy()` | ต้องรวมข้อมูลกลุ่มเดียวกันมาไว้ที่เดียว | นับยอดขายต่อจังหวัด |
| `join()` | ต้องจับคู่ข้อมูลจาก 2 ตาราง | JOIN orders กับ customers |
| `orderBy()` | ต้องเรียงข้อมูลทั้งหมด | เรียงตามราคาจากมากไปน้อย |
| `distinct()` | ต้องเช็คค่าซ้ำทั้งหมด | หาจำนวน user ไม่ซ้ำ |

### ทำไม Shuffle ถึง "แพง"?

> ⚠️ **Shuffle = การส่งข้อมูลผ่านเครือข่าย = ช้า!**

```
เครื่อง 1: [A1, B2, A3]  ──── Network ────▶  เครื่อง 1: [A1, A3, A5] (กลุ่ม A)
เครื่อง 2: [B4, A5, B6]  ──── Network ────▶  เครื่อง 2: [B2, B4, B6] (กลุ่ม B)
```

- ต้องเขียน disk ก่อนส่ง (Spill)
- ส่งผ่าน Network (ช้ากว่า RAM 1000 เท่า)
- เครื่องรับต้องรอข้อมูลครบก่อนทำงานต่อ


<div align="center">
  <img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/shuffle_concept.png?raw=1" width="800" alt="Shuffle Concept: การส่งข้อมูลข้ามเครื่องเหมือนการแยกสีลูกบอลลงถังที่ถูกต้อง">
  <br><i>Shuffle Concept: การส่งข้อมูลข้ามเครื่องเหมือนการแยกสีลูกบอลลงถังที่ถูกต้อง</i>
</div>


In [10]:
# สาธิตความแตกต่าง: คำสั่งที่มี Shuffle vs ไม่มี Shuffle
import time

# สร้างข้อมูลขนาดใหญ่ขึ้น
big_data = [(i, f"dept_{i % 50}", i * 10.5) for i in range(100000)]
big_df = spark.createDataFrame(big_data, ["id", "department", "amount"])

# ===== ไม่มี Shuffle (Narrow Transformation) =====
t0 = time.time()
result1 = big_df.filter(big_df["amount"] > 5000).select("id", "amount")
result1.count()  # trigger action
t1 = time.time()
print(f"🟢 Filter + Select (ไม่มี Shuffle): {t1-t0:.3f} วินาที")

# ===== มี Shuffle (Wide Transformation) =====
t2 = time.time()
result2 = big_df.groupBy("department").sum("amount")
result2.count()  # trigger action
t3 = time.time()
print(f"🔴 GroupBy + Sum (มี Shuffle): {t3-t2:.3f} วินาที")
print(f"\n💡 สังเกต: GroupBy ช้ากว่า Filter เพราะต้อง Shuffle!")


🟢 Filter + Select (ไม่มี Shuffle): 1.139 วินาที
🔴 GroupBy + Sum (มี Shuffle): 1.758 วินาที

💡 สังเกต: GroupBy ช้ากว่า Filter เพราะต้อง Shuffle!


# 8. 🧪 Lab: โหลดและวิเคราะห์ข้อมูลจริงด้วย Spark

### Step 1: โหลดข้อมูล CSV


In [11]:
# 1. Download Austin Crime Data (Real World Dataset)
# Dataset: Austin Crime Reports (from data.austintexas.gov)
# File size: ~500 MB
!wget -O crime.csv "https://data.austintexas.gov/api/views/fdj4-gpfu/rows.csv?accessType=DOWNLOAD"

print("✅ Downloaded crime.csv")

# 2. Load CSV into Spark DataFrame
# inferSchema=True: ให้ Spark เดา Type ของแต่ละคอลัมน์ (อาจช้าสำหรับไฟล์ใหญ่มาก)
# header=True: บรรทัดแรกคือชื่อคอลัมน์
df_crime = spark.read.csv("crime.csv", header=True, inferSchema=True)

print(f"✅ Loaded Data: {df_crime.count():,} rows")
print("\n📊 Original Schema:")
df_crime.printSchema()
print("\n📋 Sample Data:")
df_crime.show(5, truncate=False)


--2026-02-14 07:08:17--  https://data.austintexas.gov/api/views/fdj4-gpfu/rows.csv?accessType=DOWNLOAD
Resolving data.austintexas.gov (data.austintexas.gov)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.austintexas.gov (data.austintexas.gov)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘crime.csv’

crime.csv               [ <=>                ] 408.91M   692KB/s    in 15m 29s 

2026-02-14 07:23:49 (451 KB/s) - ‘crime.csv’ saved [428768699]

✅ Downloaded crime.csv
✅ Loaded Data: 2,619,292 rows

📊 Original Schema:
root
 |-- Incident Number: long (nullable = true)
 |-- Highest Offense Description: string (nullable = true)
 |-- Highest Offense Code: integer (nullable = true)
 |-- Family Violence: string (nullable = true)
 |-- Occurred Date Time: string (nullable = true)
 |-- Occurred Date: string (nullable = true)
 |-- Occurred Time: integer (nullable = true)
 |-- Report Date Time: string (null

### ⚠️ Common Pitfalls: Data Loading

| ❌ สิ่งที่ไม่ควรทำ | 💥 ผลเสีย | ✅ ทำแบบนี้แทน |
|---|---|---|
| **inferSchema=True** บนไฟล์ระดับ TB | Spark ต้องอ่านทั้งไฟล์ 1 รอบ → ช้ามาก | กำหนด `schema=struct` เอง |
| ลืม `header=True` | เอาชื่อคอลัมน์มาเป็นข้อมูลแถวแรก | `option("header", "true")` |
| ไฟล์ CSV มี Newline ในข้อความ | ข้อมูลพัง/อ่านผิดบรรทัด | `option("multiLine", "true")` |


### ⚡ Exercise 0: CSV vs Parquet

**โจทย์:** แปลงไฟล์ CSV เป็น Parquet แล้วโหลดกลับมาใหม่

**ทำไมต้อง Parquet?**
- เร็วกว่า CSV มาก (Column-oriented)
- เก็บ Schema ในตัว (ไม่ต้อง guess Type)
- ขนาดเล็กกว่า (Compression)


In [17]:
# TODO: เขียนโค้ดแปลง CSV -> Parquet
# 1. Save as Parquet (ตั้งชื่อโฟลเดอร์ crime_parquet)

df_crime.write.mode("overwrite").parquet("crime_parquet")

print("✅ Saved to crime_parquet")

# 2. Read from Parquet
df_crime_parquet = spark.read.parquet("crime_parquet")

# 3. Compare Count
print(f"✅ Loaded Parquet: {df_crime_parquet.count():,} rows")

✅ Saved to crime_parquet
✅ Loaded Parquet: 2,619,292 rows


### Step 2: Transformation (Lazy)

ลอง Filter ข้อมูลแบบง่ายๆ (ยังไม่Clean) เพื่อดูว่า Spark ทำงานแบบ Lazy Evaluation อย่างไร


In [18]:
# เลือกเฉพาะคดี 'THEFT' (ลักทรัพย์)
# หมายเหตุ: ชื่อคอลัมน์ใน Austin Dataset คือ 'Highest Offense Description'
df_theft = df_crime_parquet.filter(df_crime_parquet["Highest Offense Description"] == "THEFT")

print("✅ Transformation สร้างเสร็จ (Spark ยังไม่ทำงานจริง)")


✅ Transformation สร้างเสร็จ (Spark ยังไม่ทำงานจริง)


### Step 3: Action (Trigger)


In [19]:
import time

t0 = time.time()
count = df_theft.count()
t1 = time.time()

print(f"📊 จำนวนคดีลักทรัพย์ (THEFT): {count:,}")
print(f"⏱️ เวลาที่ใช้: {t1-t0:.2f} วินาที")


📊 จำนวนคดีลักทรัพย์ (THEFT): 239,479
⏱️ เวลาที่ใช้: 1.01 วินาที


### ⚠️ Common Pitfalls: Actions

| ❌ สิ่งที่ควรระวัง | ✅ วิธีแก้ |
|---|---|
| **`collect()` บนข้อมูลใหญ่** | Driver Memory เต็ม (OOM) | ใช้ `take(n)`, `show()`, หรือเขียนลง Disk |
| **`count()` บ่อยเกินไป** | ทำให้ Spark คำนวณใหม่ทุกรอบ (ถ้าไม่ Cache) | `count()` เท่าที่จำเป็น |


### Step 4: Execution Plan


In [20]:
df_theft.explain("simple")


== Physical Plan ==
*(1) Filter (isnotnull(Highest Offense Description#263) AND (Highest Offense Description#263 = THEFT))
+- *(1) ColumnarToRow
   +- FileScan parquet [Incident Number#262L,Highest Offense Description#263,Highest Offense Code#264,Family Violence#265,Occurred Date Time#266,Occurred Date#267,Occurred Time#268,Report Date Time#269,Report Date#270,Report Time#271,Location Type#272,Council District#273,APD Sector#274,APD District#275,Clearance Status#276,Clearance Date#277,UCR Category#278,Category Description#279,Census Block Group#280L] Batched: true, DataFilters: [isnotnull(Highest Offense Description#263), (Highest Offense Description#263 = THEFT)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/crime_parquet], PartitionFilters: [], PushedFilters: [IsNotNull(`Highest Offense Description`), EqualTo(`Highest Offense Description`,THEFT)], ReadSchema: struct<Incident Number:bigint,Highest Offense Description:string,Highest Offense Code:int,Family ...




# 8. 🧹 Deep Dive: Data Cleaning & Preparation

ข้อมูลจริงมักจะไม่ Clean! เรามาลงมือทำความสะอาดข้อมูลกัน

**ปัญหาที่พบ:**
1. ชื่อคอลัมน์ยาวและมีช่องว่าง (เช่น `Highest Offense Description`) → Rename เป็น `crime_type`
2. `Occurred Date Time` เป็น String (e.g. `01/01/2022 12:00:00 PM`) → Convert to Timestamp
3. แยก `Year`, `Month` ออกมาเพื่อวิเคราะห์ง่ายขึ้น


### 🛠️ Exercise 1: Cleaning with Spark SQL

**โจทย์:** ใช้ Spark SQL เพื่อ:
1. เปลี่ยนชื่อคอลัมน์
2. แปลงวันที่
3. กรองข้อมูลปี 2020 ขึ้นไป


In [21]:
# 1. Register Temp View
df_crime_parquet.createOrReplaceTempView("crime_raw")

# 2. Write SQL
df_clean_sql = spark.sql("""
    SELECT
        `Incident Number` as case_id,
        `Highest Offense Description` as crime_type,
        coalesce(try_to_timestamp(`Occurred Date Time`, 'MM/dd/yyyy  HH:mm'), try_to_timestamp(`Occurred Date Time`, 'MM/dd/yyyy hh:mm:ss a')) as crime_date,
        `Census Block Group` as census_block_group,
        `Council District` as district,
        `Clearance Status` as clearance_status
    FROM crime_raw
    WHERE `Occurred Date Time` IS NOT NULL
""")

df_clean_sql.printSchema()
df_clean_sql.show(5)


root
 |-- case_id: long (nullable = true)
 |-- crime_type: string (nullable = true)
 |-- crime_date: timestamp (nullable = true)
 |-- census_block_group: long (nullable = true)
 |-- district: integer (nullable = true)
 |-- clearance_status: string (nullable = true)

+-----------+--------------------+-------------------+------------------+--------+----------------+
|    case_id|          crime_type|         crime_date|census_block_group|district|clearance_status|
+-----------+--------------------+-------------------+------------------+--------+----------------+
|20112440318|THEFT BY SHOPLIFTING|2011-09-01 04:55:00|        4530024371|       5|               N|
|20112501319|BURGLARY NON RESI...|2011-09-02 03:02:00|        4910204051|       6|               N|
|20112510248|THEFT BY SHOPLIFTING|2011-09-08 04:07:00|        4910204081|       6|               N|
|20112551395|  ASSAULT BY CONTACT|2011-09-12 18:37:00|        4530002031|       9|               C|
|20115043519|    DEBIT CARD ABUSE

### 🧩 Exercise 1.2: SQL Challenge

**โจทย์:** เขียน SQL เพื่อหา **Top 3 Districts** ที่มีคดี **BURGLARY** มากที่สุด ในปี **2023**

เติมคำในช่องว่าง (`____`) ให้สมบูรณ์

In [31]:
df_crime_parquet.show(5)

+---------------+---------------------------+--------------------+---------------+------------------+-------------+-------------+-----------------+-----------+-----------+--------------------+----------------+----------+------------+----------------+--------------+------------+--------------------+------------------+
|Incident Number|Highest Offense Description|Highest Offense Code|Family Violence|Occurred Date Time|Occurred Date|Occurred Time| Report Date Time|Report Date|Report Time|       Location Type|Council District|APD Sector|APD District|Clearance Status|Clearance Date|UCR Category|Category Description|Census Block Group|
+---------------+---------------------------+--------------------+---------------+------------------+-------------+-------------+-----------------+-----------+-----------+--------------------+----------------+----------+------------+----------------+--------------+------------+--------------------+------------------+
|    20112440318|       THEFT BY SHOPLIFTIN

In [35]:
# TODO: เติมคำในช่องว่าง
sql_challenge = """
    SELECT
        `Location Type` as district,
        COUNT(*) as crime_count
    FROM crime_raw
    WHERE crime_type = 'BURGLARY'
      AND year = 2023
    GROUP BY district
    ORDER BY crime_type DESC
    LIMIT 3
"""


spark.sql(sql_challenge).show()

{"ts": "2026-02-14 07:56:51.339", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `crime_type` cannot be resolved. Did you mean one of the following? [`APD Sector`, `Location Type`, `Report Date`, `Report Time`, `Occurred Date`]. SQLSTATE: 42703", "context": {"errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o31.sql.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `crime_type` cannot be resolved. Did you mean one of the following? [`APD Sector`, `Location Type`, `Report Date`, `Report Time`, `Occurred Date`]. SQLSTATE: 42703; line 6 pos 10;\n'GlobalLimit 3\n+- 'LocalLimit 3\n   +- 'Sort ['crime_type DESC NULLS LAST], true\n      +- 'Aggregate ['district], ['Location Type AS district#489, count(1) AS crime_count#490L]\n      

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `crime_type` cannot be resolved. Did you mean one of the following? [`APD Sector`, `Location Type`, `Report Date`, `Report Time`, `Occurred Date`]. SQLSTATE: 42703; line 6 pos 10;
'GlobalLimit 3
+- 'LocalLimit 3
   +- 'Sort ['crime_type DESC NULLS LAST], true
      +- 'Aggregate ['district], ['Location Type AS district#489, count(1) AS crime_count#490L]
         +- 'Filter (('crime_type = BURGLARY) AND ('year = 2023))
            +- SubqueryAlias crime_raw
               +- View (`crime_raw`, [Incident Number#262L, Highest Offense Description#263, Highest Offense Code#264, Family Violence#265, Occurred Date Time#266, Occurred Date#267, Occurred Time#268, Report Date Time#269, Report Date#270, Report Time#271, Location Type#272, Council District#273, APD Sector#274, APD District#275, Clearance Status#276, Clearance Date#277, UCR Category#278, Category Description#279, Census Block 

In [36]:
# TODO: เติมคำในช่องว่าง
sql_challenge = """
    SELECT
        `Location Type` as district,
        COUNT(*) as crime_count
    FROM crime_raw
    WHERE `Highest Offense Description` = 'BURGLARY'
      AND YEAR(to_timestamp(`Occurred Date Time`, 'MM/dd/yyyy  HH:mm')) = 2023
    GROUP BY district
    ORDER BY crime_count DESC
    LIMIT 3
"""


spark.sql(sql_challenge).show()

+--------+-----------+
|district|crime_count|
+--------+-----------+
+--------+-----------+



### 🛠️ Exercise 2: Cleaning with PySpark Functions

**โจทย์:** ทำเหมือนข้อบน แต่ใช้ `pyspark.sql.functions` แทน SQL Strings


In [32]:
from pyspark.sql import functions as F

df_clean = df_crime_parquet.select(
    F.col("Incident Number").alias("case_id"),
    F.col("Highest Offense Description").alias("crime_type"),
    F.coalesce(F.to_timestamp("Occurred Date Time", "MM/dd/yyyy  HH:mm"), F.to_timestamp("Occurred Date Time", "MM/dd/yyyy hh:mm:ss a")).alias("crime_date"),
    F.col("Census Block Group").cast("string").alias("census_block_group"),
    F.col("Council District").alias("district"),
    F.col("Clearance Status").alias("clearance_status"),
) \
.withColumn("year", F.year("crime_date")) \
.withColumn("month", F.month("crime_date")) \
.filter(F.col("crime_date").isNotNull())

print("✅ Cleaned Data Structure:")
df_clean.printSchema()


✅ Cleaned Data Structure:
root
 |-- case_id: long (nullable = true)
 |-- crime_type: string (nullable = true)
 |-- crime_date: timestamp (nullable = true)
 |-- census_block_group: string (nullable = true)
 |-- district: integer (nullable = true)
 |-- clearance_status: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



### 🧩 Exercise 2.2: PySpark Transformations

**โจทย์:** ใช้ PySpark เพื่อกรองเฉพาะคดีที่ **Clearance Status** ไม่เป็น NULL และแสดงเฉพาะคอลัมน์ `case_id`, `crime_type`, `district`


In [41]:
# TODO: เติมคำในช่องว่าง
df_challenge = df_clean.select(
    F.col("case_id"),
    F.col("crime_type"),
    F.col("district")
).filter(
    F.col("clearance_status").isNotNull()
)

df_challenge.show(5)

+-----------+--------------------+--------+
|    case_id|          crime_type|district|
+-----------+--------------------+--------+
|20112440318|THEFT BY SHOPLIFTING|       5|
|20112501319|BURGLARY NON RESI...|       6|
|20112510248|THEFT BY SHOPLIFTING|       6|
|20112551395|  ASSAULT BY CONTACT|       9|
|20115043519|    DEBIT CARD ABUSE|       9|
+-----------+--------------------+--------+
only showing top 5 rows


In [39]:
df_clean.show(5)

+-----------+--------------------+-------------------+------------------+--------+----------------+----+-----+
|    case_id|          crime_type|         crime_date|census_block_group|district|clearance_status|year|month|
+-----------+--------------------+-------------------+------------------+--------+----------------+----+-----+
|20112440318|THEFT BY SHOPLIFTING|2011-09-01 04:55:00|        4530024371|       5|               N|2011|    9|
|20112501319|BURGLARY NON RESI...|2011-09-02 03:02:00|        4910204051|       6|               N|2011|    9|
|20112510248|THEFT BY SHOPLIFTING|2011-09-08 04:07:00|        4910204081|       6|               N|2011|    9|
|20112551395|  ASSAULT BY CONTACT|2011-09-12 18:37:00|        4530002031|       9|               C|2011|    9|
|20115043519|    DEBIT CARD ABUSE|2011-09-07 12:00:00|        4530012001|       9|               N|2011|    9|
+-----------+--------------------+-------------------+------------------+--------+----------------+----+-----+
o

### ⚠️ Common Pitfalls: Transformation

| ❌ สิ่งที่ควรระวัง | ✅ วิธีแก้ |
|---|---|
| **Chaining `withColumn`** มากเกินไป | Spark จะสร้าง plan ที่ซ้อนกันลึกมาก → ช้า | ใช้ `select()` สร้างหลายคอลัมน์ทีเดียว |
| ใช้ **Python Function (UDF)** | ช้ามาก เพราะต้องส่งข้อมูลข้ามไป Python | ใช้ **Spark Built-in Functions** (`F.xx`) เสมอ |


# 9. 📊 Analysis Exercises

ใช้ `df_clean` ที่เตรียมไว้ ตอบคำถามต่อไปนี้


In [37]:
# โจทย์ 1: 5 อันดับอาชญากรรมที่พบบ่อยที่สุด
df_clean.groupBy("crime_type").count().orderBy(F.col("count").desc()).show(5, truncate=False)


+----------------------------------+------+
|crime_type                        |count |
+----------------------------------+------+
|BURGLARY OF VEHICLE               |266983|
|THEFT                             |239478|
|FAMILY DISTURBANCE                |222524|
|CRIMINAL MISCHIEF                 |150427|
|ASSAULT WITH INJURY-FAM/DATING VIO|93653 |
+----------------------------------+------+
only showing top 5 rows


In [42]:
# โจทย์ 2: แนวโน้มอาชญากรรมรายปี (Count by Year)
df_clean.groupBy("year").count().orderBy("year").show()


+----+------+
|year| count|
+----+------+
|2003|120970|
|2004|119747|
|2005|127855|
|2006|129144|
|2007|136747|
|2008|143488|
|2009|141208|
|2010|136518|
|2011|127951|
|2012|127426|
|2013|122982|
|2014|114739|
|2015|110704|
|2016|107057|
|2017|103805|
|2018| 97932|
|2019|103075|
|2020| 98873|
|2021| 91622|
|2022| 88106|
+----+------+
only showing top 20 rows


In [43]:
# โจทย์ 3: สัดส่วนการปิดคดี (Clearance Status)
df_clean.groupBy("clearance_status").count().orderBy(F.col("count").desc()).show(truncate=False)


+----------------+-------+
|clearance_status|count  |
+----------------+-------+
|N               |1310951|
|NULL            |632684 |
|C               |592370 |
|O               |83222  |
|9               |1      |
+----------------+-------+



### 🧩 Analysis Challenge (Fill-in-the-blank)

เติมคำสั่งให้สมบูรณ์

In [ ]:
# 4. ต้องการดูค่าที่ไม่ซ้ำกันของ crime_type
df_clean.select("crime_type").______().count()

# 5. การ Join ตารางเล็ก (Lookup) กับตารางใหญ่ ควรใช้อะไรเพื่อลดการ Shuffle?
df_large.join(F.______(df_small), "id")

# 6. เมื่อเลิกใช้ DataFrame ที่ Cache ไว้ ควรคำสั่งอะไรเพื่อคืน RAM?
df.______() # hint: ตรงข้ามกับ persist

# 10. 🚀 Cache Performance Test

เปรียบเทียบความเร็วเมื่อใช้ Cache กับ DataFrame ที่ผ่านการ Clean มาแล้ว


In [ ]:
import time

# Force Transformation
df_heavy = df_clean.filter(F.col("year") >= 2020).groupBy("district").count()

# 1. No Cache
t0 = time.time()
df_heavy.show(5) # Action 1
df_heavy.count() # Action 2
t1 = time.time()
print(f"🔴 No Cache: {t1-t0:.2f} sec")

# 2. With Cache
df_clean.cache()
t2 = time.time()
df_heavy.show(5) # Action 1 (First time builds cache)
df_heavy.count() # Action 2 (Hits cache)
t3 = time.time()
print(f"🟢 Cache: {t3-t2:.2f} sec")

df_clean.unpersist()


### ⚠️ Common Pitfalls: Performance

| ❌ สิ่งที่ควรระวัง | ✅ วิธีแก้ |
|---|---|
| **ลืม `unpersist()`** | RAM เต็มโดยไม่จำเป็น | `unpersist()` เมื่อเลิกใช้ DataFrame นั้นแล้ว |
| **Cache ตารางใหญ่เกิน RAM** | Disk Spill (ช้ากว่าเดิม) | เลือก Cache เฉพาะตารางที่ใช้ซ้ำบ่อยๆ และไม่ใหญ่เกินไป |
| **Shuffle บน Key ที่เบี้ยว (Skew)** | บางเครื่องทำงานหนักตายง | ใช้ Salt Key หรือ Broadcast Join (ถ้าทำได้) |


# 11. 🗺️ Bonus: Visualizing Crime on Map

**Goal:** Plot จุดเกิดเหตุบนแผนที่จริงด้วย `folium`

**คำเตือน:** เราจะ Plot แค่ **500 จุด** ล่าสุด เพื่อป้องกัน Browser ค้าง


In [ ]:
# 1. Download Austin Census Block Groups GeoJSON
import requests
import json
import folium

geojson_url = "https://data.austintexas.gov/api/views/dwa9-qvcr/rows.geojson?accessType=DOWNLOAD"

print("⏳ Downloading GeoJSON...")
try:
    resp = requests.get(geojson_url)
    austin_geo = resp.json()
    print("✅ Downloaded GeoJSON")
except Exception as e:
    print(f"⚠️ Failed to download GeoJSON: {e}")
    austin_geo = None

if austin_geo:
    # 2. Prepare Data
    df_heatmap = df_clean.filter(F.col("census_block_group").isNotNull()) \
                         .groupBy("census_block_group").count() \
                         .toPandas()

    # DEBUG: Check values
    print("🔍 Data Sample:", df_heatmap['census_block_group'].head(5).tolist())
    print("🔍 GeoJSON Sample:", [f['properties']['geoid'] for f in austin_geo['features'][:5]])

    df_heatmap['census_block_group'] = df_heatmap['census_block_group'].astype(str)

    # 3. Create Map
    austin_map = folium.Map(location=[30.2672, -97.7431], zoom_start=11)

    folium.Choropleth(
        geo_data=austin_geo,
        name="Crime Density",
        data=df_heatmap,
        columns=["census_block_group", "count"],
        key_on="feature.properties.geoid",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.2,
        nan_fill_color="white",
        legend_name="Crime Count per Block Group"
    ).add_to(austin_map)

    folium.LayerControl().add_to(austin_map)
    austin_map


In [ ]:
folium.LayerControl().add_to(austin_map)
austin_map

# 12. 📌 สรุป (Summary)

## สิ่งที่เรียนรู้วันนี้

| หัวข้อ | สรุป |
|--------|------|
| **เครื่องเดียวไม่พอ** | เมื่อข้อมูล > RAM/CPU → ต้องใช้ Distributed Computing |
| **Spark คืออะไร** | Distributed Compute Engine (ไม่ใช่ DB, ไม่ใช่ Storage) |
| **Architecture** | Driver (สมอง) + Executors (แรงงาน) + Cluster Manager (ผู้จัดสรร) |
| **Lazy Evaluation** | Transformation สร้างแผน, Action สั่งทำ → ช่วย optimize |
| **DAG** | กราฟแสดงลำดับการทำงาน → Catalyst Optimizer ปรับให้เร็ว |
| **Shuffle** | ส่งข้อมูลข้ามเครื่อง → ช้า แพง ต้องระวัง |
| **Cache** | เก็บ DataFrame ใน RAM สำหรับใช้ซ้ำ → ประหยัดเวลา |
| **explain()** | ดูแผนการทำงาน → เข้าใจว่า Spark จะทำอะไร |

## 🔮 สัปดาห์หน้า: Data Pipeline

> สัปดาห์ที่ 5: กระบวนการจัดการข้อมูล (Data Pipeline)  
> ingest → clean → transform → store → analyze

---

## 📚 แหล่งเรียนรู้เพิ่มเติม

- [Apache Spark Documentation](https://spark.apache.org/docs/latest/)
- [Spark: The Definitive Guide](https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/)
- [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/)


In [ ]:
# ปิด Spark Session
# spark.stop() # Comment ออกก่อนเพื่อทำ Lab Test ต่อ
print("✅ จบเนื้อหาหลัก! ไปลุย Lab Test กันต่อ 🚀")


# 13. 🧪 Lab Test: Olist E-Commerce Analytics (Real World Data)

**Dataset:** Brazilian E-Commerce Public Dataset by Olist  
**Source:** [Kaggle / GitHub](https://github.com/ayushic2899/Brazilian-E-Commerce-Public-Dataset-by-Olist)  
**Goal:** วิเคราะห์ยอดขายและพฤติกรรมลูกค้าจากข้อมูลจริง 3 ตาราง (Orders, Items, Products)

### 📥 Step 0: Download Data


In [ ]:
# Download Dataset from GitHub
!wget -q -O olist_orders.csv https://raw.githubusercontent.com/ayushic2899/Brazilian-E-Commerce-Public-Dataset-by-Olist/master/olist_orders_dataset.csv
!wget -q -O olist_items.csv https://raw.githubusercontent.com/ayushic2899/Brazilian-E-Commerce-Public-Dataset-by-Olist/master/olist_order_items_dataset.csv
!wget -q -O olist_products.csv https://raw.githubusercontent.com/ayushic2899/Brazilian-E-Commerce-Public-Dataset-by-Olist/master/olist_products_dataset.csv
!wget -q -O olist_customers.csv https://raw.githubusercontent.com/ayushic2899/Brazilian-E-Commerce-Public-Dataset-by-Olist/master/olist_customers_dataset.csv

print("✅ Download Cleaned!")

### 🛠️ Task 1: Load & Clean Data

1. อ่านไฟล์ CSV ทั้ง 4 ไฟล์เข้าสู่ Spark DataFrame
2. ตรวจสอบ Schema และจัดการ Type (ถ้าจำเป็น)
3. สร้าง Temp View เพื่อเตรียมพร้อมสำหรับ SQL


In [ ]:
# TODO: Load Data
# df_orders = ...
# df_items = ...
# df_products = ...
# df_customers = ...


### 🔗 Task 2: Join Data

จงสร้าง `df_master` โดยการ Join ตารางดังนี้:
1. `orders` JOIN `items` (ON order_id)
2. JOIN `products` (ON product_id)
3. JOIN `customers` (ON customer_id)

> **Tip:** ตรวจสอบจำนวนแถวหลัง Join ว่าเพิ่มขึ้นหรือลดลงผิดปกติหรือไม่


In [ ]:
# TODO: Join Data
# df_master = ...


### 📊 Task 3: Analytics

ตอบคำถามต่อไปนี้ (เลือกใช้ SQL หรือ PySpark ก็ได้):

1. **Top Products:** สินค้าหมวดไหน (`product_category_name`) มียอดขายรวม (price) สูงที่สุด 5 อันดับแรก?
2. **Top Cities:** เมืองไหน (`customer_city`) มีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
3. **(Optional) Monthly Sales:** ยอดขายรวมแต่ละเดือนเป็นอย่างไร? (แนวโน้ม)


In [ ]:
# TODO: Analytics Code


In [ ]:
# ปิด Spark Session หลังทำ Lab Test เสร็จ
spark.stop()